<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/k2_pix_small.png">
*This notebook contains instructional material from the [K2 Guest Observer Office](https://keplerscience.arc.nasa.gov/); the content is available [on GitHub](https://github.com/gully/k2-metadata).*


<!--NAVIGATION-->
< [K2 TPF header index analysis](02.02-TPF-header_analysis.ipynb) | [Contents](Index.ipynb) | [Other K2 metadata sources](03.00-Other-metadata-sources.ipynb) >

# EPIC and GOs merge

This notebook merges the GO proposals available on the [Kepler/K2 Guest Observer Office Approve Programs website](https://keplerscience.arc.nasa.gov/k2-approved-programs.html) with the EPIC catalog [available on MAST](https://archive.stsci.edu/pub/k2/catalogs/).  The resulting table contains additional EPIC metadata on the targets proposed by Guest Observers.  Custom Apertures will not possess this additional metadata, but remain included for completeness.

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
GO_df = pd.read_csv('../metadata/tidy/GO_proposal_metadata.csv', 
                    dtype={'campaign':str})

/Users/obsidian/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Hack because something is screwed up in my k2-target index

In [4]:
import feather

In [5]:
EPIC_df = feather.read_dataframe('../metadata/EPIC_catalog/EPIC_15cols_27Feb2018.feather')

Naturally, all of EPIC is much larger than subset that has-been / will-be targeted by Guest Observers.

In [6]:
GO_df.shape, EPIC_df.shape

((558217, 6), (50550332, 15))

In [7]:
EPIC_df.head()

,id,hip,tyc,ucac,twomass,sdss,objtype,kepflag,stpropflag,k2_ra,k2_dec,kp,nomad,mflg,prox
0,201000001.0,NaN,None,None,11370387-1048329,None,STAR,JHK,None,174.266139,-10.809154,18.558001,None,None,NaN
1,201000002.0,NaN,None,None,11372639-1048232,None,STAR,J,None,174.359970,-10.806450,18.257000,None,None,NaN
2,201000003.0,NaN,None,396-052344,11390735-1048225,None,STAR,BV,pmV,174.780561,-10.806233,16.306000,None,None,NaN
3,201000004.0,NaN,None,None,11372709-1048189,None,STAR,JHK,None,174.362902,-10.805261,18.348000,None,None,NaN
4,201000005.0,NaN,None,396-052314,11380583-1048162,None,STAR,gri,pmV,174.524379,-10.804541,15.894000,None,None,NaN


In [8]:
GO_df.head()

,EPIC ID,RA (J2000) [deg],Dec (J2000) [deg],magnitude,Investigation IDs,campaign
0,200000811,,,,LC_M35_TILE,0
1,200000812,,,,LC_M35_TILE,0
2,200000813,,,,LC_M35_TILE,0
3,200000814,,,,LC_M35_TILE,0
4,200000815,,,,LC_M35_TILE,0


In [9]:
%%time
df = pd.merge(GO_df, EPIC_df, left_on='EPIC ID', right_on='id', how='left', indicator=True)

CPU times: user 33.7 s, sys: 1min, total: 1min 34s
Wall time: 1min 49s


In [10]:
df._merge.value_counts()

both          378540
left_only     179677
right_only         0
Name: _merge, dtype: int64

The "left_only" indicates sources that are proposed, but not in the EPIC.  That's Okay, since guest observers come up with custom targets, like asteroids, faint galaxies, saturated stars, fast-moving objects, etc.  Let's verify that the largest EPIC ID in this "left only" subset is less than 201 million, the threshold EPIC ID reserved for custom apertures.

In [11]:
vec = df[df._merge == 'left_only']['EPIC ID']
int_mask = vec.apply(type) == np.int
vec[int_mask].max()

200235137

Indeed, we're safe from missing targets.  Healthy sanity-check! :)

In [13]:
df.columns

Index(['EPIC ID', 'RA (J2000) [deg]', 'Dec (J2000) [deg]', 'magnitude',
       'Investigation IDs', 'campaign', 'id', 'hip', 'tyc', 'ucac', 'twomass',
       'sdss', 'objtype', 'kepflag', 'stpropflag', 'k2_ra', 'k2_dec', 'kp',
       'nomad', 'mflg', 'prox', '_merge'],
      dtype='object')

In [14]:
df.iloc[300000]

EPIC ID                         245980230
RA (J2000) [deg]               353.095655
Dec (J2000) [deg]              -10.002241
magnitude                          14.996
Investigation IDs              GO12123_LC
campaign                               12
id                             2.4598e+08
hip                                   NaN
tyc                                  None
ucac                           400-138795
twomass                  23322294-1000078
sdss                 1666-301-3-0146-0031
objtype                              STAR
kepflag                               gri
stpropflag                            rpm
k2_ra                             353.096
k2_dec                           -10.0022
kp                                 14.996
nomad                                None
mflg                  AAA-222-111-000-0-0
prox                                 25.3
_merge                               both
Name: 300000, dtype: object

Looks good!  Let's drop the `_merge` keyword and save to disk.

In [15]:
df.shape

(558217, 22)

In [16]:
df = df[df.columns[~df.columns.isin(['_merge'])]]

In [17]:
df.shape

(558217, 21)

Uncomment below to overwrite

In [20]:
#df.to_csv('../metadata/tidy/GO_EPIC_merge_uptoC19.csv', index=False)

In [18]:
! du -hs ../metadata/tidy/GO_EPIC_merge_uptoC19.csv

 84M	../metadata/tidy/GO_EPIC_merge_uptoC19.csv


In [19]:
! head ../metadata/tidy/GO_EPIC_merge_uptoC19.csv

EPIC ID,RA (J2000) [deg],Dec (J2000) [deg],magnitude,Investigation IDs,campaign,id,hip,tyc,ucac,twomass,sdss,objtype,kepflag,stpropflag,k2_ra,k2_dec,kp,nomad,mflg,prox
200000811, , , , LC_M35_TILE,0,,,,,,,,,,,,,,,
200000812, , , , LC_M35_TILE,0,,,,,,,,,,,,,,,
200000813, , , , LC_M35_TILE,0,,,,,,,,,,,,,,,
200000814, , , , LC_M35_TILE,0,,,,,,,,,,,,,,,
200000815, , , , LC_M35_TILE,0,,,,,,,,,,,,,,,
200000816, , , , LC_M35_TILE,0,,,,,,,,,,,,,,,
200000817, , , , LC_M35_TILE,0,,,,,,,,,,,,,,,
200000818, , , , LC_M35_TILE,0,,,,,,,,,,,,,,,
200000819, , , , LC_M35_TILE,0,,,,,,,,,,,,,,,


In [ ]:
#df.to_feather('../metadata/tidy/GO_EPIC_C00-19.feather')

<!--NAVIGATION-->
< [K2 TPF header index analysis](02.02-TPF-header_analysis.ipynb) | [Contents](Index.ipynb) | [Other K2 metadata sources](03.00-Other-metadata-sources.ipynb) >